In [1]:
# Verificar o arquivo juros_selic
!hdfs dfs -cat /user/ivan/data/juros_selic/juros_selic

"data";"valor"
"01/06/1986";"1,27"
"01/07/1986";"1,95"
"01/08/1986";"2,57"
"01/09/1986";"2,94"
"01/10/1986";"1,96"
"01/11/1986";"2,37"
"01/12/1986";"5,47"
"01/01/1987";"11,00"
"01/02/1987";"19,61"
"01/03/1987";"11,95"
"01/04/1987";"15,30"
"01/05/1987";"24,63"
"01/06/1987";"18,02"
"01/07/1987";"8,91"
"01/08/1987";"8,09"
"01/09/1987";"7,99"
"01/10/1987";"9,45"
"01/11/1987";"12,92"
"01/12/1987";"14,38"
"01/01/1988";"16,78"
"01/02/1988";"18,35"
"01/03/1988";"16,59"
"01/04/1988";"20,25"
"01/05/1988";"18,65"
"01/06/1988";"20,17"
"01/07/1988";"24,69"
"01/08/1988";"22,63"
"01/09/1988";"26,25"
"01/10/1988";"29,79"
"01/11/1988";"28,41"
"01/12/1988";"30,24"
"01/01/1989";"22,97"
"01/02/1989";"18,95"
"01/03/1989";"20,41"
"01/04/1989";"11,52"
"01/05/1989";"11,43"
"01/06/1989";"27,29"
"01/07/1989";"33,15"
"01/08/1989";"35,49"
"01/09/1989";"38,58"
"01/10/1989";"47,70"
"01/11/1989";"48,41"
"01/12/1989";"64,21"
"01/01/1990";"67,60"
"01/02/1990";"82,04"
"01/03/1990";"36,76"
"01/04/1990";"4,23"
"01/05/199

In [24]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
# Criando um DF
juros = spark.read.csv('/user/ivan/data/juros_selic/juros_selic',
                             header="True", sep=";")

In [4]:
# Visualizando o df 
juros.show(5)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
+----------+-----+
only showing top 5 rows



In [9]:
# Convertendo a coluna data para timestamp
data_padrao = juros.withColumn('data', from_unixtime(unix_timestamp(col("data"), "dd/mm/yyyy"), 'MM/dd/yyyy'))

# Visualizando
data_padrao.show(5)

+----------+-----+
|      data|valor|
+----------+-----+
|01/01/1986| 1,27|
|01/01/1986| 1,95|
|01/01/1986| 2,57|
|01/01/1986| 2,94|
|01/01/1986| 1,96|
+----------+-----+
only showing top 5 rows



In [14]:
# Criar o campo ano no DF
ano_unix = data_padrao.withColumn('ano_unix', from_unixtime(unix_timestamp(col("data"), "dd/mm/yyyy"), 'yyyy'))
ano_unix.show(5)

+----------+-----+--------+
|      data|valor|ano_unix|
+----------+-----+--------+
|01/01/1986| 1,27|    1986|
|01/01/1986| 1,95|    1986|
|01/01/1986| 2,57|    1986|
|01/01/1986| 2,94|    1986|
|01/01/1986| 1,96|    1986|
+----------+-----+--------+
only showing top 5 rows



In [15]:
# Cria a coluno ano str utilizando a função substring
ano_substring = ano_unix.withColumn('ano_str', substring(col('data'),7,4))
ano_substring.show(5)

+----------+-----+--------+-------+
|      data|valor|ano_unix|ano_str|
+----------+-----+--------+-------+
|01/01/1986| 1,27|    1986|   1986|
|01/01/1986| 1,95|    1986|   1986|
|01/01/1986| 2,57|    1986|   1986|
|01/01/1986| 2,94|    1986|   1986|
|01/01/1986| 1,96|    1986|   1986|
+----------+-----+--------+-------+
only showing top 5 rows



In [16]:
# Cria a coluna ano split com a função split
ano_split = ano_substring.withColumn('ano_split', split(col('data'), '/').getItem(2))
ano_split.show(5)

+----------+-----+--------+-------+---------+
|      data|valor|ano_unix|ano_str|ano_split|
+----------+-----+--------+-------+---------+
|01/01/1986| 1,27|    1986|   1986|     1986|
|01/01/1986| 1,95|    1986|   1986|     1986|
|01/01/1986| 2,57|    1986|   1986|     1986|
|01/01/1986| 2,94|    1986|   1986|     1986|
|01/01/1986| 1,96|    1986|   1986|     1986|
+----------+-----+--------+-------+---------+
only showing top 5 rows



In [17]:
# Salvar o df com o cabeçalho
ano_split.write.csv('/user/ivan/juros_selic', header='true')

In [18]:
# Verificando o arquivo no diretorio
!hdfs dfs -ls /user/ivan/juros_selic

Found 2 items
-rw-r--r--   2 root supergroup          0 2022-04-10 20:25 /user/ivan/juros_selic/_SUCCESS
-rw-r--r--   2 root supergroup      13086 2022-04-10 20:25 /user/ivan/juros_selic/part-00000-50038b19-419d-4a12-bd6b-88784d8d9f61-c000.csv


In [19]:
# verificando o conteudo do arquivo salvo
!hdfs dfs -cat /user/ivan/juros_selic/part-00000-50038b19-419d-4a12-bd6b-88784d8d9f61-c000.csv

data,valor,ano_unix,ano_str,ano_split
01/01/1986,"1,27",1986,1986,1986
01/01/1986,"1,95",1986,1986,1986
01/01/1986,"2,57",1986,1986,1986
01/01/1986,"2,94",1986,1986,1986
01/01/1986,"1,96",1986,1986,1986
01/01/1986,"2,37",1986,1986,1986
01/01/1986,"5,47",1986,1986,1986
01/01/1987,"11,00",1987,1987,1987
01/01/1987,"19,61",1987,1987,1987
01/01/1987,"11,95",1987,1987,1987
01/01/1987,"15,30",1987,1987,1987
01/01/1987,"24,63",1987,1987,1987
01/01/1987,"18,02",1987,1987,1987
01/01/1987,"8,91",1987,1987,1987
01/01/1987,"8,09",1987,1987,1987
01/01/1987,"7,99",1987,1987,1987
01/01/1987,"9,45",1987,1987,1987
01/01/1987,"12,92",1987,1987,1987
01/01/1987,"14,38",1987,1987,1987
01/01/1988,"16,78",1988,1988,1988
01/01/1988,"18,35",1988,1988,1988
01/01/1988,"16,59",1988,1988,1988
01/01/1988,"20,25",1988,1988,1988
01/01/1988,"18,65",1988,1988,1988
01/01/1988,"20,17",1988,1988,1988
01/01/1988,"24,69",1988,1988,1988
01/01/1988,"22,63",1988,1988,1988
01/01/1988,"26,25",1988,1988,1988
01/01/1988,"29,79",19

# Exercicios 2

In [20]:
# Criando um DF
juros_selic = spark.read.csv('/user/ivan/data/juros_selic/juros_selic',
                             header="True", sep=";")
juros_selic.show(5)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
+----------+-----+
only showing top 5 rows



In [22]:
# Cria a coluna ano split com a função split
juros_ano = juros_selic.withColumn('ano', split(col('data'), '/').getItem(2))
juros_ano.show(5)

+----------+-----+----+
|      data|valor| ano|
+----------+-----+----+
|01/06/1986| 1,27|1986|
|01/07/1986| 1,95|1986|
|01/08/1986| 2,57|1986|
|01/09/1986| 2,94|1986|
|01/10/1986| 1,96|1986|
+----------+-----+----+
only showing top 5 rows



In [26]:
# Alterando o a coluna valor para float
juros_valor = juros_ano.withColumn('valor', regexp_replace(col('valor'), '\,','\.').cast(FloatType()))
juros_valor.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: float (nullable = true)
 |-- ano: string (nullable = true)



In [34]:
# Agrupar os dados do df com as agregações de media, valor e ordenação por ano
juros_agrupado = juros_valor.groupBy('ano').agg(count('ano').alias('meses')
                                               ,format_number(avg('valor'),2).alias('valor_medio')
                                               ,min('valor').alias('valor_minimo')
                                               ,max('valor').alias('valor_maximo')
                                               ).sort(desc('ano'))
juros_agrupado.show()

+----+-----+-----------+------------+------------+
| ano|meses|valor_medio|valor_minimo|valor_maximo|
+----+-----+-----------+------------+------------+
|2019|    2|       0.46|        0.37|        0.54|
|2018|   12|       0.52|        0.47|        0.58|
|2017|   12|       0.79|        0.54|        1.09|
|2016|   12|       1.10|         1.0|        1.22|
|2015|   12|       1.04|        0.82|        1.18|
|2014|   12|       0.87|        0.77|        0.96|
|2013|   12|       0.66|        0.49|        0.81|
|2012|   12|       0.68|        0.54|        0.89|
|2011|   12|       0.92|        0.84|        1.07|
|2010|   12|       0.78|        0.59|        0.93|
|2009|   12|       0.79|        0.66|        1.05|
|2008|   12|       0.98|         0.8|        1.18|
|2007|   12|       0.94|         0.8|        1.08|
|2006|   12|       1.18|        0.99|        1.43|
|2005|   12|       1.46|        1.22|        1.66|
|2004|   12|       1.26|        1.08|        1.48|
|2003|   12|       1.76|       

In [35]:
# Salvar o DF em formato orc
juros_agrupado.write.orc('/user/ivan/relatorio_anual', compression='zlib')

In [36]:
# Checando o arquivo
!hdfs dfs -ls /user/ivan/relatorio_anual

Found 35 items
-rw-r--r--   2 root supergroup          0 2022-04-10 20:43 /user/ivan/relatorio_anual/_SUCCESS
-rw-r--r--   2 root supergroup        580 2022-04-10 20:43 /user/ivan/relatorio_anual/part-00000-9edd2494-6c31-48b1-affe-a4499b9df4ee-c000.zlib.orc
-rw-r--r--   2 root supergroup        580 2022-04-10 20:43 /user/ivan/relatorio_anual/part-00001-9edd2494-6c31-48b1-affe-a4499b9df4ee-c000.zlib.orc
-rw-r--r--   2 root supergroup        584 2022-04-10 20:43 /user/ivan/relatorio_anual/part-00002-9edd2494-6c31-48b1-affe-a4499b9df4ee-c000.zlib.orc
-rw-r--r--   2 root supergroup        571 2022-04-10 20:43 /user/ivan/relatorio_anual/part-00003-9edd2494-6c31-48b1-affe-a4499b9df4ee-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2022-04-10 20:43 /user/ivan/relatorio_anual/part-00004-9edd2494-6c31-48b1-affe-a4499b9df4ee-c000.zlib.orc
-rw-r--r--   2 root supergroup        580 2022-04-10 20:43 /user/ivan/relatorio_anual/part-00005-9edd2494-6c31-48b1-affe-a4499b9df4ee-c000.zlib.orc
-r